In [1]:
# 数值特征
import pandas as pd
import numpy as np
import json
from tqdm import tqdm as tqdm
from sklearn.model_selection import KFold

In [2]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        #train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item
# 补充别名实体进入kb

            
name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = sorted(list(set(name_id[id])))

In [3]:
# 提取candidate候选特征
kb_column = []
train_column = []
text_id = []
label_colum = []
num_attrs = []
num_abstract_words = []
num_alias = []
m_id = 0
m_id_list = []
num_candidate = []
equal_subject = []
shuminghao = [] 
entity_common = []
len_mention = []
mention_start = []
numerical_f = []
is_eng = []
for s in tqdm(train_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        if m['mention'] not in name_id : # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        if m['kb_id'] != 'nil':
            name_list = [kb[m['kb_id']]['subject']]+kb[m['kb_id']]['alias']
            if m['mention'] not in name_list:
                continue 
        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                #candi_text += predicate['predicate']

                if predicate['predicate'] == '摘要':
                    candidate_abstract = predicate['object']
                    candidate_abstract_numwords = len(candidate_abstract)
#                 if predicate['predicate'] == '标签':
#                     candidate_label += predicate['object']
            if m_candidate_id==m['kb_id']:
                label = 1
            else:
                label = 0
            train_column.append(m['mention'])
            kb_column.append(m_candidate_id)
            text_id.append(s['text_id'])
            label_colum.append(label)
            num_attrs.append(candidate_numattrs)
            num_abstract_words.append(candidate_abstract_numwords)
            num_alias.append(len(set(candidate_detail['alias']+[candidate_detail['subject']])))
            m_id_list.append(m_id)

        m_id += 1
        
data = pd.DataFrame()
data['text_id'] = text_id
data['kb_id'] = kb_column
data['train_mention'] = train_column
data['label'] = label_colum
data['m_id'] = m_id_list

100%|██████████| 90000/90000 [00:13<00:00, 6460.24it/s]


In [4]:
# 流行度特征
# 总流行度
data_ctr = []
kfold = KFold(n_splits=9,shuffle=False,random_state=2019)
for train_index,test_index in kfold.split(data):
    print(test_index)
    stat_train_data,stat_test_data = data.loc[train_index,:],data.loc[test_index,:]
    #print(test_data.head())
    train_group = stat_train_data.groupby('kb_id',as_index=False)['label'].agg({'label_mean':'mean','label_max':'max',
                                                                                'label_var':'var','label_count':'count'})
    stat_test_data = stat_test_data.merge(train_group,on='kb_id',how='left')
    print(stat_test_data[stat_test_data.label_mean.isnull()].shape)
    train_group = stat_train_data.groupby(['kb_id','train_mention'])['label'].agg({'m_label_mean':'mean','m_label_max':'max',
                                                                                   'm_label_var':'var','m_label_count':'count'})
    stat_test_data = stat_test_data.merge(train_group,on=['kb_id','train_mention'],how='left')

    # mean and var
    to_rank = [ 'm_label_mean', 'm_label_max',
           'm_label_var', 'm_label_count', 
              ]
    for f in tqdm(to_rank):
        data_group = stat_test_data.groupby('kb_id')[f].agg({'kb_'+f+'_var':'var','kb_'+f+'_mean':'mean','kb_'+f+'_max':'max',
                                                       'kb_'+f+'_min':'min'})
        stat_test_data = stat_test_data.merge(data_group,on='kb_id',how='left')

    # rank 特征
    to_rank = [ 'm_label_mean', 'm_label_max',
           'm_label_var', 'm_label_count', 
              ]
    for f in to_rank:
        data_group = stat_test_data.groupby('kb_id')[f].rank(ascending=False)
        stat_test_data['kbid_rank_'+f] = data_group
    
    data_ctr.append(stat_test_data)
data_ctr = pd.concat(data_ctr,axis=0)


[     0      1      2 ... 174073 174074 174075]
(19791, 9)


/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
  0%|          | 0/4 [00:00<?, ?it/s]/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
100%|██████████| 4/4 [00:01<00:00,  3.58it/s]


[174076 174077 174078 ... 348149 348150 348151]
(19651, 9)


100%|██████████| 4/4 [00:01<00:00,  3.08it/s]


[348152 348153 348154 ... 522225 522226 522227]
(18949, 9)


...

100%|██████████| 4/4 [00:01<00:00,  3.37it/s]


[1218530 1218531 1218532 ... 1392602 1392603 1392604]
(19438, 9)


100%|██████████| 4/4 [00:01<00:00,  3.08it/s]


[1392605 1392606 1392607 ... 1566677 1566678 1566679]
(19570, 9)


100%|██████████| 4/4 [00:01<00:00,  3.09it/s]


In [5]:
# rank 特征
to_rank = ['label_mean', 'label_max',
       'label_var', 'label_count', 'm_label_mean', 'm_label_max',
       'm_label_var', 'm_label_count','kb_m_label_mean_var',
       'kb_m_label_mean_mean', 'kb_m_label_mean_max', 'kb_m_label_mean_min',
       'kb_m_label_max_var', 'kb_m_label_max_mean', 'kb_m_label_max_max',
       'kb_m_label_max_min', 'kb_m_label_var_var', 'kb_m_label_var_mean',
       'kb_m_label_var_max', 'kb_m_label_var_min', 'kb_m_label_count_var',
       'kb_m_label_count_mean', 'kb_m_label_count_max', 'kb_m_label_count_min',
       'kbid_rank_m_label_mean', 'kbid_rank_m_label_max',
       'kbid_rank_m_label_var', 'kbid_rank_m_label_count'
          ]
for f in to_rank:
    data_group = data_ctr.groupby('m_id')[f].rank(ascending=False)
    data_ctr['mid_rank_'+f] = data_group
    try:
        data_ctr['mid_rank_'+f] = data_ctr['mid_rank_'+f].astype('int32')
    except:
        pass

In [6]:
# mean and var
to_rank = [ 'label_mean', 'label_max',
       'label_var', 'label_count', 'm_label_mean', 'm_label_max',
       'm_label_var', 'm_label_count'
          ]
for f in tqdm(to_rank):
    data_group = data_ctr.groupby('m_id')[f].agg({'mid_'+f+'_var':'var','mid_'+f+'_mean':'mean',
                                                  'mid_'+f+'_max':'max','mid_'+f+'_min':'min'})
    data_ctr = data_ctr.merge(data_group,on='m_id',how='left')

  0%|          | 0/8 [00:00<?, ?it/s]/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
100%|██████████| 8/8 [00:11<00:00,  1.53s/it]


In [7]:
# ctr 特征

In [8]:
for item in data_ctr.columns:
    if data_ctr.dtypes[item]=='float64':
        data_ctr[item] = data_ctr[item].astype('float32')

In [9]:
data_ctr.reset_index(drop=True,inplace=True)
data_ctr.to_pickle('data/step2.pkl')

In [10]:
test_data = pd.read_pickle('data/step1_test.pkl').loc[:,['text_id','kb_id','train_mention','m_id','label']]

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [11]:
test_data.shape

(764217, 5)

In [12]:
# 流行度特征
# 总流行度


train_group = data.groupby('kb_id',as_index=False)['label'].agg({'label_mean':'mean','label_max':'max',
                                                                            'label_var':'var','label_count':'count'})
test_data = test_data.merge(train_group,on='kb_id',how='left')
print(test_data[test_data.label_mean.isnull()].shape)

train_group = data.groupby(['kb_id','train_mention'])['label'].agg({'m_label_mean':'mean','m_label_max':'max',
                                                                               'm_label_var':'var','m_label_count':'count'})
test_data = test_data.merge(train_group,on=['kb_id','train_mention'],how='left')

# mean and var
to_rank = [ 'm_label_mean', 'm_label_max',
       'm_label_var', 'm_label_count', 
          ]
for f in tqdm(to_rank):
    data_group = test_data.groupby('kb_id')[f].agg({'kb_'+f+'_var':'var','kb_'+f+'_mean':'mean','kb_'+f+'_max':'max',
                                                   'kb_'+f+'_min':'min'})
    test_data = test_data.merge(data_group,on='kb_id',how='left')

# rank 特征
to_rank = [ 'm_label_mean', 'm_label_max',
       'm_label_var', 'm_label_count', 
          ]
for f in to_rank:
    data_group = test_data.groupby('kb_id')[f].rank(ascending=False)
    test_data['kbid_rank_'+f] = data_group


(42625, 9)


/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
  0%|          | 0/4 [00:00<?, ?it/s]/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


In [13]:
test_data.shape

(764217, 33)

In [14]:
# rank 特征
to_rank = ['label_mean', 'label_max',
       'label_var', 'label_count', 'm_label_mean', 'm_label_max',
       'm_label_var', 'm_label_count','kb_m_label_mean_var',
       'kb_m_label_mean_mean', 'kb_m_label_mean_max', 'kb_m_label_mean_min',
       'kb_m_label_max_var', 'kb_m_label_max_mean', 'kb_m_label_max_max',
       'kb_m_label_max_min', 'kb_m_label_var_var', 'kb_m_label_var_mean',
       'kb_m_label_var_max', 'kb_m_label_var_min', 'kb_m_label_count_var',
       'kb_m_label_count_mean', 'kb_m_label_count_max', 'kb_m_label_count_min',
       'kbid_rank_m_label_mean', 'kbid_rank_m_label_max',
       'kbid_rank_m_label_var', 'kbid_rank_m_label_count'
          ]
for f in to_rank:
    data_group = test_data.groupby('m_id')[f].rank(ascending=False)
    test_data['mid_rank_'+f] = data_group
# mean and var
to_rank = [ 'label_mean', 'label_max',
       'label_var', 'label_count', 'm_label_mean', 'm_label_max',
       'm_label_var', 'm_label_count'
          ]
for f in tqdm(to_rank):
    data_group = test_data.groupby('m_id')[f].agg({'mid_'+f+'_var':'var','mid_'+f+'_mean':'mean',
                                                  'mid_'+f+'_max':'max','mid_'+f+'_min':'min'})
    test_data = test_data.merge(data_group,on='m_id',how='left')

  0%|          | 0/8 [00:00<?, ?it/s]/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
100%|██████████| 8/8 [00:05<00:00,  1.47it/s]


In [15]:
test_data.reset_index(drop=True,inplace=True)
test_data.to_pickle('data/step2_test.pkl')